# Ignore all the bs below, used it to build my stochastic models. I'll let you know when the important stuff begins

In [1]:
data1 = CSV.read("sept19_filter_agg_scen1.csv")

UndefVarError: UndefVarError: CSV not defined

In [15]:
start_stations = Set(unique(data1[!,2]))
end_stations = Set(unique(data1[!,3]))
stations = collect(union(start_stations,end_stations))
time_intervals = unique(data1[!,8])
days = unique(data1[!,7]);

In [16]:
#define sizes 
# m = total number of start station clusters
m= size(stations,1)
# n = total number of time windows
n = size(time_intervals,1)
# o = total number of days
o = size(days,1);


In [17]:
r_ride = 1.15
c_bike = 345;

# Optimization Model

In [20]:
function optimize_distribution(data)
    #initialize model
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0,Int)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)


    #solve model
    status = solve(mod)
    
    return sum(getvalue(x)[:,1,1]),getvalue(d),getobjectivevalue(mod)
end



optimize_distribution (generic function with 1 method)

In [21]:
optimize_distribution(data1)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1870531 rows, 1836000 columns and 7088950 nonzeros
Model fingerprint: 0x8997650e
Variable types: 36000 continuous, 1800000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 1463.0000000
Presolve removed 1852443 rows and 1746517 columns (presolve time = 5s) ...
Presolve removed 1852443 rows and 1746528 columns
Presolve time: 8.84s
Presolved: 18088 rows, 89472 columns, 249573 nonzeros
Variable types: 18058 continuous, 71414 integer (53421 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       

(564.0, [-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

...

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

...

[-0.0 1.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; 0.0 0.0 … -0.0 0.0; 0.0 0.0 … 0.0 -0.0]

[-0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 

# Baseline Model

In [75]:
function baseline_distribution(min_bikes,data)
    uniform_dist = round(min_bikes/m)
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,l=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,[i=1:m,d=1:o],x[i,1,d]==uniform_dist)
#     @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)

    status = solve(mod)
    
    return getvalue(d)
end


baseline_distribution (generic function with 1 method)

# Everything Above is Useless, The Real Shit Starts from Here

Instructions to run this baby:
1. Make sure your files are named properly and that you have the csv files for all 4 scenarios for the months you want to run the model for. The file naming convention is "mmmmyy_filter_agg_scenx.csv" .For example, "sept19_filter_agg_scen1.csv"
2. Every csv file should have the following columns with this exact naming convention: "start_area", "end_area", "dep_flow","Hour"(0-23),"Day"(1-30/31).
3. Run the functions below and then call the compute_everything function. The arguments to this function are 
    - a list of months you want to compute everything. For example - ["sept19","may18","oct20"]. Every month in this list should match the way the month is spelt in the dataset. 
    - The acceptable total loss per month (eg 200000)
    - Operational Revenue - Operational costs (we discussed this number to be 1.15)
    - Fixed cost of a bike (we discussed this to be 200)
    - example call of the function - main_result = compute_everything(["sept19"],200000,1.15,200)
4. The function ouputs the following:
    - A CSV file with the optimal number of bikes per station per day for each month in the list of months
    - A CSV file with the optimal model allowed flows for Scenario 1.
    - A dataframe with the following information for each month (each row of the dataframe corresponds to one month in the list of months: Minimum Number of Bikes from the optimal model, Number of missed rides for the optimal and baseline scenarios, number of met rides for the optimal and baseline scenarios.

In [1]:
using JuMP, Gurobi, DataFrames, CSV, Plots

# Stochastic Model

In [2]:
function compute_missed_rides(data,d)
    missed_rides = [(data[i,:dep_flow]-d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]])
    for i=1:size(data,1)]
    return sum(missed_rides)
end

compute_missed_rides (generic function with 1 method)

In [22]:
#### stochastic optimization model
function stochastic_model(data1,data2,data3,data4,acceptable_loss,r_ride,c_bike)
    
    # m = total number of start station clusters
    m= size(unique(data1[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data1[!,:Hour]),1)
    # o = total number of days
    o = size(unique(data1[!,:Day]),1)
    
    #1. initialize model
    
    mod = Model(solver = GurobiSolver())
    
    #2. decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o,s=1:4]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o,s=1:4] >= 0,Int)
    
    #3. objective function
    @objective(mod,Max,sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4)
    
    #4. constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o,s=1:4],x[i,t,l,s] - sum(d[i,j,t,l,s] for j=1:m) + sum(d[j,i,t,l,s] for j=1:m)-x[i,t+1,l,s]==0)
    @constraint(mod,[d=1:o,s=1:4], sum(x[i,1,d,s] for i=1:m) == sum(x[i,1,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data1,1)],d[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day],1] <= data1[i,:dep_flow])
    @constraint(mod,[i=1:size(data2,1)],d[data2[i,:start_area],data2[i,:end_area],data2[i,:Hour]+1,data2[i,:Day],2] <= data2[i,:dep_flow])
    @constraint(mod,[i=1:size(data3,1)],d[data3[i,:start_area],data3[i,:end_area],data3[i,:Hour]+1,data3[i,:Day],3] <= data3[i,:dep_flow])
    @constraint(mod,[i=1:size(data4,1)],d[data4[i,:start_area],data4[i,:end_area],data4[i,:Hour]+1,data4[i,:Day],4] <= data4[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o,s=1:4],d[i,i,t,l,s]==0)
    @constraint(mod,r_ride*(sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4) - c_bike*sum(x[i,1,1,1] for i=1:m) >= -acceptable_loss)
    
    #5. solve model
    status = solve(mod)
    
    #6. Compute missed rides and average it for the 4 scenarios
    missed_rides1 = compute_missed_rides(data1,getvalue(d)[:,:,:,:,1])
    missed_rides2 = compute_missed_rides(data2,getvalue(d)[:,:,:,:,2])
    missed_rides3 = compute_missed_rides(data3,getvalue(d)[:,:,:,:,3])
    missed_rides4 = compute_missed_rides(data4,getvalue(d)[:,:,:,:,4])
    avg_missed_rides = (missed_rides1+missed_rides2+missed_rides3+missed_rides4)/4
    
    #7. Compute number of rides met and average it for each scenario
    avg_met_rides = getobjectivevalue(mod)
    
    #8. Input number of bikes per station per day into a dataframe (only for scenario 1)
    bikes = getvalue(x)[:,1,:,1]
    x_df = DataFrame(Station=Int[],Day=Int[],Num_Bikes=Int[])
    for i=1:m
        for j=1:o
            push!(x_df,[i,j,bikes[i,j]])
        end
    end
    
    
    #9. Input allowed flows from scenario 1 into its dataframe (data1)
    flows= getvalue(d)[:,:,:,:,1]
    allowed_flows = []
    for i in 1:size(data1,1)
        append!(allowed_flows,flows[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day]])
    end
    data1[:allowed_flows] = allowed_flows
    final_flows_data = data1[:,[2,3,4,7,8,9]]
    
   
    
    #10. return
    return sum(getvalue(x)[:,1,1,1]),avg_missed_rides,avg_met_rides,x_df,final_flows_data
    
end
    

stochastic_model (generic function with 1 method)

# Stochastic Baseline 

In [23]:
function stochastic_baseline(data1,data2,data3,data4,min_bikes,r_ride,c_bike)
    # m = total number of start station clusters
    m= size(unique(data1[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data1[!,:Hour]),1)
    # o = total number of days
    o = size(unique(data1[!,:Day]),1)
    
    #1. uniform_dist = uniformly distributed number of bikes at each station
    uniform_dist = round(min_bikes/m)
    
    #2. initialize model
    
    mod = Model(solver = GurobiSolver())
    
    #3. decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o,s=1:4]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o,s=1:4] >= 0,Int)
    
    #4. objective function
    @objective(mod,Max,sum(d[i,j,t,l,s] for i=1:m,j=1:m,t=1:n,l=1:o,s=1:4)/4)
    
    
    #5. constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o,s=1:4],x[i,t,l,s] - sum(d[i,j,t,l,s] for j=1:m) + sum(d[j,i,t,l,s] for j=1:m)-x[i,t+1,l,s]==0)
    @constraint(mod,[d=1:o,s=1:4], sum(x[i,1,d,s] for i=1:m) == sum(x[i,1,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data1,1)],d[data1[i,:start_area],data1[i,:end_area],data1[i,:Hour]+1,data1[i,:Day],1] <= data1[i,:dep_flow])
    @constraint(mod,[i=1:size(data2,1)],d[data2[i,:start_area],data2[i,:end_area],data2[i,:Hour]+1,data2[i,:Day],2] <= data2[i,:dep_flow])
    @constraint(mod,[i=1:size(data3,1)],d[data3[i,:start_area],data3[i,:end_area],data3[i,:Hour]+1,data3[i,:Day],3] <= data3[i,:dep_flow])
    @constraint(mod,[i=1:size(data4,1)],d[data4[i,:start_area],data4[i,:end_area],data4[i,:Hour]+1,data4[i,:Day],4] <= data4[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o,s=1:4],d[i,i,t,l,s]==0)
    @constraint(mod,[i=1:m,d=1:o,s=1:4],x[i,1,d,s]==uniform_dist)
    
    #6. solve model
    status = solve(mod)
    
    #7. Compute missed rides and average it for the 4 scenarios
    missed_rides1 = compute_missed_rides(data1,getvalue(d)[:,:,:,:,1])
    missed_rides2 = compute_missed_rides(data2,getvalue(d)[:,:,:,:,2])
    missed_rides3 = compute_missed_rides(data3,getvalue(d)[:,:,:,:,3])
    missed_rides4 = compute_missed_rides(data4,getvalue(d)[:,:,:,:,4])
    avg_missed_rides = (missed_rides1+missed_rides2+missed_rides3+missed_rides4)/4
    
    #8. Compute number of rides met and average it for each scenario
    avg_met_rides = getobjectivevalue(mod)
   
    
    #9. return
    return avg_missed_rides,avg_met_rides
    
end
    

stochastic_baseline (generic function with 1 method)

# Main Function That Computes Everything

In [24]:
function compute_everything(month_list,acceptable_loss,r_ride,c_bike)
    main_df = DataFrame(Month=String[],Min_Bikes=Int[],Missed_Optimal=Int[],
    Missed_Baseline=Int[],Met_Optimal=Int[],Met_Baseline=Int[])
    for i in month_list
        
        #read in datasets for the 4 scenarios
        
        data1 = CSV.read("$i"*"_filter_agg_scen1.csv")
        data2 = CSV.read("$i"*"_filter_agg_scen2.csv")
        data3 = CSV.read("$i"*"_filter_agg_scen3.csv")
        data4 = CSV.read("$i"*"_filter_agg_scen4.csv")
        
        #define sizes - do it for one dataset because its the same for all 4 scenarios
        
        # m = total number of start station clusters
        m= size(unique(data1[!,:start_area]),1)
        # n = total number of time windows
        n = size(unique(data1[!,:Hour]),1)
        # o = total number of days
        o = size(unique(data1[!,:Day]),1)
        
        #run optimal model and get outputs
        optimal_solution = stochastic_model(data1,data2,data3,data4,acceptable_loss,r_ride,c_bike)
        Min_Bikes = optimal_solution[1]
        Missed_Optimal = optimal_solution[2]
        Met_Optimal = optimal_solution[3]
#         CSV.write("$i"*"_bike_distribution.csv",optimal_solution[4])
#         CSV.write("$i"*"_flows.csv",optimal_solution[5])
        
        #run baseline model and get outputs
        baseline_solution = stochastic_baseline(data1,data2,data3,data4,Min_Bikes,r_ride,c_bike)
        Missed_Baseline = baseline_solution[1]
        Met_Baseline = baseline_solution[2]
        
        
        #push values into main dataframe
        push!(main_df,[i,Min_Bikes,
                round(Missed_Optimal),round(Missed_Baseline),round(Met_Optimal),round(Met_Baseline)])
        
    end
    
    return main_df
end

compute_everything (generic function with 1 method)

In [25]:
main_result = compute_everything(["jan15", "feb15", "mar15", "may15", "jan16", "feb16", "mar16", "may16", "jan17", "feb17", "mar17", "may17"],200000,1.15,200)
CSV.write("2015_2016_2017_main_result.csv",main_result)

Checkpoint1

BoundsError: BoundsError: attempt to access 16×16×24×31×4 Array{Variable,5} at index [1, 17, 1, 1, 1]

In [52]:
for month in ["jan15", "feb15", "mar15", "may15", "jan16", "feb16", "mar16", "may16", "jan17", "feb17", "mar17", "may17"]
    data1 = CSV.read(month*"_filter_agg_scen1.csv")
    data2 = CSV.read(month*"_filter_agg_scen2.csv")
    data3 = CSV.read(month*"_filter_agg_scen3.csv")
    data4 = CSV.read(month*"_filter_agg_scen4.csv")
    
    s1 = size(unique(data1[!,:start_area]),1)
    s2 = size(unique(data2[!,:start_area]),1)
    s3 = size(unique(data3[!,:start_area]),1)
    s4 = size(unique(data4[!,:start_area]),1)
    
    e1 = size(unique(data1[!,:end_area]),1)
    e2 = size(unique(data1[!,:end_area]),1)
    e3 = size(unique(data1[!,:end_area]),1)
    e4 = size(unique(data1[!,:end_area]),1)
    
    if s1 == s2 & s1 == s3 & s1 == s4 & s1 == e1 & s1 == e2 & s1 == e3 & s1 == e4
        println(month*"  ALL GOOD")
    else
        println(month*"  NOmT ALL GOOD")
    end
end

jan15  ALL GOOD
feb15  ALL GOOD
mar15  ALL GOOD
may15  ALL GOOD
jan16  ALL GOOD
feb16  ALL GOOD
mar16  ALL GOOD
may16  ALL GOOD
jan17  ALL GOOD
feb17  ALL GOOD
mar17  ALL GOOD
may17  ALL GOOD


In [47]:
data2 = CSV.read("jan15"*"_filter_agg_scen2.csv")
size(unique(data2[!,:end_area]),1)

50

In [48]:
data1 = CSV.read("jan15"*"_filter_agg_scen1.csv")
size(unique(data1[!,:end_area]),1)

50

In [49]:
data3 = CSV.read("jan15"*"_filter_agg_scen3.csv")
size(unique(data3[!,:end_area]),1)

50

In [50]:
data2 = CSV.read("jan15"*"_filter_agg_scen2.csv")
size(unique(data2[!,:start_area]),1)

50

In [51]:
unique(data2[!,:end_area])

50-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50

In [42]:
unique(data2[!,:start_area])

16-element Array{Int64,1}:
  1
  3
  6
  7
  8
  9
 11
 17
 19
 24
 26
 28
 30
 34
 37
 45

# Prediction optimization

In [62]:
using JuMP, Gurobi, DataFrames, CSV, Plots

In [63]:
function compute_missed_rides_pred(data,d)
    missed_rides = [(data[i,:dep_flow]-d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1])
    for i=1:size(data,1)]
    return sum(missed_rides)
end

compute_missed_rides_pred (generic function with 1 method)

In [64]:
predictions = CSV.read("sep19_filter_agg_scen1.csv")
predictions = predictions[(predictions.Day .== 1), :]
num_bikes = 625;

In [65]:
sum(predictions[!,:dep_flow])

4331

In [66]:
function optimize_distribution(data, num_bikes)
    
    # m = total number of start station clusters
    m= size(unique(data[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data[!,:Hour]),1)

    #initialize model
    mod = Model(solver = GurobiSolver())

    #decision variables: x is the allocation of bikes and d the allocated flows
    @variable(mod,x[i=1:m,t=1:n] >= 0, Int)
    @variable(mod,d[i=1:m,j=1:m,t=1:n] >= 0, Int)

    #objective function
    @objective(mod,Max,sum(d[i,j,t] for i=1:m,j=1:m,t=1:n))
    
    ### Constraints
    # Flow conservation
    @constraint(mod,[i=1:m,t=1:n-1],x[i,t] - sum(d[i,j,t] for j=1:m) + sum(d[j,i,t] for j=1:m)-x[i,t+1]==0)
    
    # Smaller than demand
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1] <= data[i,:dep_flow])
    
    # No transfer inside stations
#     @constraint(mod,[i=1:m,t=1:n],d[i,i,t] == 0)
    # Total number of bikes below the number of bought bikes
    @constraint(mod,sum(x[i,1] for i=1:m) <= num_bikes)
    
    #solve model
    status = solve(mod)
    
    return sum(getvalue(x)[:,1,1]),getvalue(d),getobjectivevalue(mod)
end

optimize_distribution (generic function with 1 method)

In [184]:
function pred_optimization(source_file, num_bikes)
    
    data = CSV.read(source_file)
    
    # m = total number of start station clusters
    m= size(unique(data[!,:start_area]),1)
    # n = total number of time windows
    n = size(unique(data[!,:Hour]),1)

    #initialize model
    mod = Model(solver = GurobiSolver())

    #decision variables: x is the allocation of bikes and d the allocated flows
    @variable(mod,x[i=1:m,t=1:n] >= 0, Int)
    @variable(mod,d[i=1:m,j=1:m,t=1:n] >= 0, Int)
    @variable(mod,max_stat)

    #objective function
    @objective(mod,Max,sum(d[i,j,t] for i=1:m,j=1:m,t=1:n) - 0.0001*max_stat)
    
    ### Constraints
    # Flow conservation
    @constraint(mod,[i=1:m,t=1:n-1], x[i,t] - sum(d[i,j,t] for j=1:m) + sum(d[j,i,t] for j=1:m)-x[i,t+1]==0)
    
    # Smaller than demand
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1] <= data[i,:dep_flow])
    
    # Not more than 50 bikes in the morning per station
#     @constraint(mod,[i=1:m], x[i,1] <= 40)
    
    # Constraint to minimize the maximum number of bikes per station
    @constraint(mod, [i=1:m], max_stat >= x[i,1])
    
    # No transfer inside stations
    @constraint(mod,[i=1:m,t=1:n],d[i,i,t] == 0)
    
    # Total number of bikes below the number of bought bikes
    @constraint(mod, sum(x[i,1] for i=1:m) <= num_bikes)
    
    #solve model
    status = solve(mod)
    
    
    
    ### Get results
    
    # Aggregate statistics
    bikes_used = sum(getvalue(x)[:,1])
    pred_rides = sum(data[!,:dep_flow])
    missed_rides = compute_missed_rides_pred(data,getvalue(d)[:,:,:])
    met_rides = getobjectivevalue(mod)
    
    #8. Input number of bikes per station per day into a dataframe (only for scenario 1)
    bikes = getvalue(x)[:,1]
    x_df = DataFrame(Station=Int[],Num_Bikes=Int[])
    for i=1:m
        push!(x_df,[i,bikes[i]])
    end
    
    #9. Input allowed flows from scenario 1 into its dataframe (data1)
    flows= getvalue(d)[:,:,:]
    allowed_flows = []
    for i in 1:size(data,1)
        append!(allowed_flows,flows[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1])
    end
    data[:allowed_flows] = allowed_flows
    
    
    
    #9. return
    return bikes_used, pred_rides, missed_rides, met_rides, x_df, data
    
end


pred_optimization (generic function with 1 method)

In [185]:
source_file = "synth_1_up.csv"
num_bikes = 625
bikes_used, pred_rides, missed_rides, met_rides, x_df, data = pred_optimization(source_file, num_bikes)

IJulia.clear_output()

bikes_used, pred_rides, missed_rides, met_rides

(625.0, 21757, 4873.0, 16883.9966)

In [186]:
sum(x_df[!,:Num_Bikes])

625

In [187]:
print(x_df)

50×2 DataFrame
│ Row │ Station │ Num_Bikes │
│     │ Int64   │ Int64     │
├─────┼─────────┼───────────┤
│ 1   │ 1       │ 0         │
│ 2   │ 2       │ 6         │
│ 3   │ 3       │ 0         │
│ 4   │ 4       │ 34        │
│ 5   │ 5       │ 0         │
│ 6   │ 6       │ 0         │
│ 7   │ 7       │ 34        │
│ 8   │ 8       │ 0         │
│ 9   │ 9       │ 7         │
│ 10  │ 10      │ 3         │
│ 11  │ 11      │ 17        │
│ 12  │ 12      │ 28        │
│ 13  │ 13      │ 28        │
│ 14  │ 14      │ 10        │
│ 15  │ 15      │ 34        │
│ 16  │ 16      │ 0         │
│ 17  │ 17      │ 12        │
│ 18  │ 18      │ 0         │
│ 19  │ 19      │ 11        │
│ 20  │ 20      │ 0         │
│ 21  │ 21      │ 0         │
│ 22  │ 22      │ 27        │
│ 23  │ 23      │ 34        │
│ 24  │ 24      │ 34        │
│ 25  │ 25      │ 3         │
│ 26  │ 26      │ 34        │
│ 27  │ 27      │ 31        │
│ 28  │ 28      │ 0         │
│ 29  │ 29      │ 0         │
│ 30  │ 30      │ 0      

In [75]:
names(data)

15-element Array{Symbol,1}:
 :X1           
 :start_area   
 :end_area     
 :Hour         
 :avg_temp     
 :avg_hmd      
 :avg_drct     
 :avg_wind     
 :sum_pcpt     
 :avg_press    
 :avg_vis      
 :max_gust     
 :weekday      
 :dep_flow     
 :allowed_flows

In [77]:
sum(data[(data.start_area .== 24), :][!,:dep_flow]), sum(data[(data.start_area .== 24), :][!,:allowed_flows])

(785, 593.0)

In [78]:
sum(data[(data.end_area .== 24), :][!,:dep_flow]), sum(data[(data.end_area .== 24), :][!,:allowed_flows])

(364, 364.0)

# Analysis of stochastic output

In [152]:
x = CSV.read("may19_bike_distribution.csv")
println(sum(x[(x.Day .== 1), :][!, :Num_Bikes]))
print(x)

547
1550×3 DataFrame
│ Row  │ Station │ Day   │ Num_Bikes │
│      │ Int64   │ Int64 │ Int64     │
├──────┼─────────┼───────┼───────────┤
│ 1    │ 1       │ 1     │ 31        │
│ 2    │ 1       │ 2     │ 18        │
│ 3    │ 1       │ 3     │ 7         │
│ 4    │ 1       │ 4     │ 9         │
│ 5    │ 1       │ 5     │ 6         │
│ 6    │ 1       │ 6     │ 8         │
│ 7    │ 1       │ 7     │ 42        │
│ 8    │ 1       │ 8     │ 19        │
│ 9    │ 1       │ 9     │ 20        │
│ 10   │ 1       │ 10    │ 2         │
│ 11   │ 1       │ 11    │ 27        │
│ 12   │ 1       │ 12    │ 6         │
│ 13   │ 1       │ 13    │ 5         │
│ 14   │ 1       │ 14    │ 14        │
│ 15   │ 1       │ 15    │ 25        │
│ 16   │ 1       │ 16    │ 30        │
│ 17   │ 1       │ 17    │ 0         │
│ 18   │ 1       │ 18    │ 9         │
│ 19   │ 1       │ 19    │ 5         │
│ 20   │ 1       │ 20    │ 19        │
│ 21   │ 1       │ 21    │ 21        │
│ 22   │ 1       │ 22    │ 23        │
│ 23

│ 345  │ 12      │ 4     │ 4         │
│ 346  │ 12      │ 5     │ 5         │
│ 347  │ 12      │ 6     │ 3         │
│ 348  │ 12      │ 7     │ 1         │
│ 349  │ 12      │ 8     │ 1         │
│ 350  │ 12      │ 9     │ 6         │
│ 351  │ 12      │ 10    │ 6         │
│ 352  │ 12      │ 11    │ 14        │
│ 353  │ 12      │ 12    │ 1         │
│ 354  │ 12      │ 13    │ 6         │
│ 355  │ 12      │ 14    │ 0         │
│ 356  │ 12      │ 15    │ 0         │
│ 357  │ 12      │ 16    │ 0         │
│ 358  │ 12      │ 17    │ 5         │
│ 359  │ 12      │ 18    │ 3         │
│ 360  │ 12      │ 19    │ 0         │
│ 361  │ 12      │ 20    │ 1         │
│ 362  │ 12      │ 21    │ 3         │
│ 363  │ 12      │ 22    │ 0         │
│ 364  │ 12      │ 23    │ 11        │
│ 365  │ 12      │ 24    │ 14        │
│ 366  │ 12      │ 25    │ 8         │
│ 367  │ 12      │ 26    │ 6         │
│ 368  │ 12      │ 27    │ 8         │
│ 369  │ 12      │ 28    │ 5         │
│ 370  │ 12      │ 29    

│ 741  │ 24      │ 28    │ 4         │
│ 742  │ 24      │ 29    │ 5         │
│ 743  │ 24      │ 30    │ 7         │
│ 744  │ 24      │ 31    │ 6         │
│ 745  │ 25      │ 1     │ 8         │
│ 746  │ 25      │ 2     │ 0         │
│ 747  │ 25      │ 3     │ 3         │
│ 748  │ 25      │ 4     │ 6         │
│ 749  │ 25      │ 5     │ 4         │
│ 750  │ 25      │ 6     │ 7         │
│ 751  │ 25      │ 7     │ 1         │
│ 752  │ 25      │ 8     │ 3         │
│ 753  │ 25      │ 9     │ 2         │
│ 754  │ 25      │ 10    │ 20        │
│ 755  │ 25      │ 11    │ 4         │
│ 756  │ 25      │ 12    │ 12        │
│ 757  │ 25      │ 13    │ 3         │
│ 758  │ 25      │ 14    │ 0         │
│ 759  │ 25      │ 15    │ 6         │
│ 760  │ 25      │ 16    │ 0         │
│ 761  │ 25      │ 17    │ 0         │
│ 762  │ 25      │ 18    │ 8         │
│ 763  │ 25      │ 19    │ 15        │
│ 764  │ 25      │ 20    │ 17        │
│ 765  │ 25      │ 21    │ 20        │
│ 766  │ 25      │ 22    

│ 1152 │ 38      │ 5     │ 9         │
│ 1153 │ 38      │ 6     │ 7         │
│ 1154 │ 38      │ 7     │ 1         │
│ 1155 │ 38      │ 8     │ 2         │
│ 1156 │ 38      │ 9     │ 4         │
│ 1157 │ 38      │ 10    │ 3         │
│ 1158 │ 38      │ 11    │ 1         │
│ 1159 │ 38      │ 12    │ 2         │
│ 1160 │ 38      │ 13    │ 3         │
│ 1161 │ 38      │ 14    │ 4         │
│ 1162 │ 38      │ 15    │ 0         │
│ 1163 │ 38      │ 16    │ 1         │
│ 1164 │ 38      │ 17    │ 7         │
│ 1165 │ 38      │ 18    │ 4         │
│ 1166 │ 38      │ 19    │ 0         │
│ 1167 │ 38      │ 20    │ 1         │
│ 1168 │ 38      │ 21    │ 2         │
│ 1169 │ 38      │ 22    │ 4         │
│ 1170 │ 38      │ 23    │ 2         │
│ 1171 │ 38      │ 24    │ 0         │
│ 1172 │ 38      │ 25    │ 0         │
│ 1173 │ 38      │ 26    │ 7         │
│ 1174 │ 38      │ 27    │ 2         │
│ 1175 │ 38      │ 28    │ 2         │
│ 1176 │ 38      │ 29    │ 12        │
│ 1177 │ 38      │ 30    

In [155]:
println(x[(x.Num_Bikes .>= 100), :])

30×3 DataFrame
│ Row │ Station │ Day   │ Num_Bikes │
│     │ Int64   │ Int64 │ Int64     │
├─────┼─────────┼───────┼───────────┤
│ 1   │ 8       │ 15    │ 221       │
│ 2   │ 8       │ 21    │ 119       │
│ 3   │ 9       │ 25    │ 152       │
│ 4   │ 14      │ 12    │ 409       │
│ 5   │ 14      │ 18    │ 152       │
│ 6   │ 14      │ 20    │ 183       │
│ 7   │ 14      │ 31    │ 123       │
│ 8   │ 29      │ 2     │ 252       │
│ 9   │ 29      │ 14    │ 279       │
│ 10  │ 29      │ 17    │ 275       │
│ 11  │ 29      │ 28    │ 283       │
│ 12  │ 33      │ 23    │ 151       │
│ 13  │ 33      │ 27    │ 161       │
│ 14  │ 39      │ 3     │ 244       │
│ 15  │ 39      │ 4     │ 244       │
│ 16  │ 39      │ 5     │ 277       │
│ 17  │ 39      │ 6     │ 194       │
│ 18  │ 39      │ 7     │ 189       │
│ 19  │ 39      │ 8     │ 177       │
│ 20  │ 39      │ 9     │ 141       │
│ 21  │ 39      │ 10    │ 200       │
│ 22  │ 39      │ 11    │ 145       │
│ 23  │ 39      │ 13    │ 230      